In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import os
import numpy as np

print(sklearn.__version__)

0.19.0


In [2]:
# Divide in train and test files [80:20] 

# Directory having content

doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1045
262


In [3]:
# Build Tagged dataset to be fed in nltk hmm tagger
# It is a list of tagged sentences. 
# training_data = [ [(word, tag), (word, tag).....]
# [(word, tag), (word, tag).....]
#]

# Directory having tags
tag_dir = 'Data/new_tags'

training_data = []

for f in train_file_list:
    training_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            training_sentences.append(pairs_in_line)
    if len(training_sentences) > 0:
        training_data.extend(training_sentences)

print training_data[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [4]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
    ]
    if i > 0:
        word1 = sent[i-1][0]
        tag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
           # '-1:tag1=' + tag1,
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2words(sent):
    return [word for word, tag in sent] 

In [5]:
print training_data[0]
print sent2features(training_data[0])[0]
print sent2tags(training_data[0])[0]
print sent2words(training_data[0])[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [6]:
x_train = [sent2features(s) for s in training_data]
y_train = [sent2tags(s) for s in training_data]

In [7]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

In [8]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [9]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [10]:
# Train and save the model
trainer.train('osint-bomb-blast.crfsuite')

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('osint-bomb-blast.crfsuite')

In [12]:
# Create test data
test_data = []
for f in test_file_list:
    test_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            test_sentences.append(pairs_in_line)
    if len(test_sentences) > 0:
        test_data.extend(test_sentences)

print test_data[0]

[('An', 'O'), ('explosion', 'O'), ('in', 'O'), ('a', 'O'), ('bus', 'O'), ('carrying', 'O'), ('employees', 'O'), ('of', 'O'), ('the', 'O'), ('state', 'ORG_Others'), ('secretariat', 'ORG_Others'), ('of', 'O'), ('Khyber', 'LOC_Others'), ('Pakhtunkhwa', 'LOC_Others'), ('(', 'LOC_Others'), ('KPK', 'LOC_Others'), (')', 'LOC_Others'), ('government', 'ORG_Others'), ('on', 'O'), ('Friday', 'O'), ('afternoon', 'O'), ('killed', 'O'), ('at', 'O'), ('least', 'O'), ('19', 'O'), ('of', 'O'), ('them', 'O'), ('and', 'O'), ('injured', 'O'), ('45', 'O'), (',', 'O'), ('according', 'O'), ('to', 'O'), ('Shiraz', 'PER_Others'), ('Paracha', 'PER_Others'), (',', 'O'), ('spokesperson', 'O'), ('for', 'O'), ('the', 'O'), ('KPK', 'LOC_Others'), ('Chief', 'O'), ('Minister', 'O'), ('.', 'O'), ('This', 'O'), ('comes', 'O'), ('after', 'O'), ('Sundays', 'O'), ('Peshawar', 'LOC_Event'), ('church', 'O'), ('blast', 'O'), ('that', 'O'), ('claimed', 'O'), ('80', 'O'), ('lives', 'O'), ('.', 'O'), ('Police', 'ORG_Others'), ('

In [13]:
example_sent = test_data[0]
print(' '.join(sent2words(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2tags(example_sent)))

An explosion in a bus carrying employees of the state secretariat of Khyber Pakhtunkhwa ( KPK ) government on Friday afternoon killed at least 19 of them and injured 45 , according to Shiraz Paracha , spokesperson for the KPK Chief Minister . This comes after Sundays Peshawar church blast that claimed 80 lives . Police said a timer device was used to set off the bomb which could have been placed in a bag in the bus . Mr. Paracha told The Hindu on the phone that the bus was ferrying more than 60 government employees back home when the incident happened , 15 km away from Peshawar . Meena Menon OPEN
('Predicted:', 'O O O O O O O O O O O O LOC_Event LOC_Event O O O O O O O O O O O O O O O O O O O PER_Others PER_Others O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O PER_Others O ORG_Others ORG_Others O O O O O O O O O O O O O O O O O O O O O O O O O O O O O')
('Correct:  ', 'O O O O O O O O O ORG_Others ORG_Others O LOC_Others LOC_Others LOC_Others 

In [14]:
def report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [15]:
x_test = [sent2features(s) for s in test_data]
y_test = [sent2tags(s) for s in test_data]

In [16]:
%%time
y_pred = [tagger.tag(xseq) for xseq in x_test]

CPU times: user 1.02 s, sys: 8 ms, total: 1.03 s
Wall time: 1.03 s


In [17]:
print(report(y_test, y_pred))

             precision    recall  f1-score   support

    Accused       0.00      0.00      0.00        65
      Event       0.00      0.00      0.00        81
LOC_Accused       0.22      0.02      0.04       100
  LOC_Assoc       0.00      0.00      0.00        81
  LOC_Event       0.52      0.39      0.44      1287
 LOC_Others       0.59      0.58      0.58      1860
 LOC_Victim       0.00      0.00      0.00        30
   Location       0.33      0.01      0.02       129
ORG_Accused       0.66      0.62      0.64       517
  ORG_Assoc       0.00      0.00      0.00        15
 ORG_Others       0.70      0.58      0.64      2463
 ORG_Victim       0.33      0.03      0.05        68
PER_Accused       0.58      0.40      0.47       679
  PER_Assoc       0.00      0.00      0.00        66
 PER_Others       0.77      0.75      0.76      2080
 PER_Victim       0.33      0.16      0.21       309
     Victim       0.00      0.00      0.00        16

avg / total       0.61      0.53      0.56  

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
num_test_data = len(y_test)
print len(y_pred)
print num_test_data

261
261


In [19]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER_Others'] = 0
actual_tag_count['PER_Others'] = 0
matched_tag_count['PER_Victim'] = 0
actual_tag_count['PER_Victim'] = 0
matched_tag_count['PER_Accused'] = 0
actual_tag_count['PER_Accused'] = 0
matched_tag_count['ORG_Victim'] = 0
actual_tag_count['ORG_Victim'] = 0
matched_tag_count['ORG_Accused'] = 0
actual_tag_count['ORG_Accused'] = 0
matched_tag_count['ORG_Others'] = 0
actual_tag_count['ORG_Others'] = 0
matched_tag_count['LOC_Accused'] = 0
actual_tag_count['LOC_Accused'] = 0
matched_tag_count['LOC_Others'] = 0
actual_tag_count['LOC_Others'] = 0
matched_tag_count['LOC_Event'] = 0
actual_tag_count['LOC_Event'] = 0
matched_tag_count['LOC_Victim'] = 0
actual_tag_count['LOC_Victim'] = 0

for i in xrange(num_test_data):
    y_test_i = y_test[i]
    y_pred_i = y_pred[i]
    test_tag_length = len(y_test[i])
    predicted_tag_length = len(y_pred[i])
    for j in xrange(test_tag_length):
        if y_test_i[j] in actual_tag_count.keys():
            actual_tag_count[y_test_i[j]] = actual_tag_count[y_test_i[j]] + 1
        
        if y_test_i[j] == y_pred_i[j]:
            if y_test_i[j] in matched_tag_count.keys():
                matched_tag_count[y_test_i[j]] = matched_tag_count[y_test_i[j]] + 1

In [20]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER_Others: '+str(matched_tag_count['PER_Others'])+ ' ' + str(actual_tag_count['PER_Others'])+ ' ' + str(matched_tag_count['PER_Others']*100/actual_tag_count['PER_Others'])+'%')
print ('PER_Victim: '+str(matched_tag_count['PER_Victim'])+ ' ' + str(actual_tag_count['PER_Victim'])+ ' '+str(matched_tag_count['PER_Victim']*100/actual_tag_count['PER_Victim'])+'%')
print ('PER_Accused: '+str(matched_tag_count['PER_Accused'])+ ' ' + str(actual_tag_count['PER_Accused'])+ ' '+str(matched_tag_count['PER_Accused']*100/actual_tag_count['PER_Accused'])+'%')
print ('ORG_Victim: '+str(matched_tag_count['ORG_Victim']) + ' '+ str(actual_tag_count['ORG_Victim'])+ ' '+str(matched_tag_count['ORG_Victim']*100/actual_tag_count['ORG_Victim'])+'%')
print ('ORG_Accused: '+str(matched_tag_count['ORG_Accused']) + ' '+ str(actual_tag_count['ORG_Accused'])+ ' '+str(matched_tag_count['ORG_Accused']*100/actual_tag_count['ORG_Accused'])+'%')
print ('ORG_Others: '+str(matched_tag_count['ORG_Others']) + ' '+ str(actual_tag_count['ORG_Others'])+ ' '+str(matched_tag_count['ORG_Others']*100/actual_tag_count['ORG_Others'])+'%')
print ('LOC_Accused: '+str(matched_tag_count['LOC_Accused'])+ ' ' + str(actual_tag_count['LOC_Accused'])+ ' '+str(matched_tag_count['LOC_Accused']*100/actual_tag_count['LOC_Accused'])+'%')
print ('LOC_Others: '+str(matched_tag_count['LOC_Others'])+ ' ' + str(actual_tag_count['LOC_Others'])+ ' '+str(matched_tag_count['LOC_Others']*100/actual_tag_count['LOC_Others'])+'%')
print ('LOC_Event: '+str(matched_tag_count['LOC_Event']) + ' '+ str(actual_tag_count['LOC_Event'])+ ' '+str(matched_tag_count['LOC_Event']*100/actual_tag_count['LOC_Event'])+'%')
print ('LOC_Victim: '+str(matched_tag_count['LOC_Victim']) + ' '+ str(actual_tag_count['LOC_Victim'])+ ' '+str(matched_tag_count['LOC_Victim']*100/actual_tag_count['LOC_Victim'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER_Others: 1562 2080 75%
PER_Victim: 48 309 15%
PER_Accused: 270 679 39%
ORG_Victim: 2 68 2%
ORG_Accused: 323 517 62%
ORG_Others: 1436 2463 58%
LOC_Accused: 2 100 2%
LOC_Others: 1079 1860 58%
LOC_Event: 498 1287 38%
LOC_Victim: 0 30 0%
